In [6]:
# this is the code for Part 1, which is the prerequisite to create the Toronto map
import numpy as np 
import json 
from geopy.geocoders import Nominatim
!pip install folium
import folium 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
required_cols = ['Postcode', 'Borough','Neighbourhood']
for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(required_cols)))=="True"):
        pstl_data_df = pd.DataFrame(table)    
    break
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()
pstl_data_df = pstl_data_df[pstl_data_df.Borough!="Not assigned"]
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()
pstl_data_df['new_nghbr'] = np.where(pstl_data_df['Neighbourhood']=='Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighbourhood'])
pstl_data_df.head(10)

can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postcode','Borough'])['new_nghbr'].apply(','.join).reset_index())
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postcode": "PostalCode","new_nghbr":"Neighborhood"})

can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postcode','Borough'])['new_nghbr'].apply(','.join).reset_index())
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postcode": "PostalCode","new_nghbr":"Neighborhood"})

     |████████████████████████████████| 92kB 12.1MB/s eta 0:00:01
Libraries imported.
Shape of Dataframe is -  (287, 3)
Shape of Dataframe is -  (210, 3)


In [7]:
# download libraries
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: done

# All requested packages already installed.

Installation Done!
Geo Coder imported!


In [8]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [9]:
can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'] = zip(*can_postal_cd_df['PostalCode'].apply(get_geocoder))
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'], can_postal_cd_df['PostalCode'], can_postal_cd_df['Borough'], can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

The geograpical coordinates of Toronto, Ontario are 43.653963, -79.387207.
